In [1]:
%cd ..

/Users/Matteo/PycharmProjects/TrafficEmu


# Bayesian optimization: finding the minimum emission

In this notebook we will performing bayesian optimization on the emulator trained on our sumo simulation. We are interested in identifying the parameters which enable us to obtain the minimum travel time. Our emulator is a gaussian process with an **rbf** kernel.


## Imports

In [30]:
import pickle
import numpy as np
import emukit as ek
import GPy
import pandas as pd

from emukit.model_wrappers import GPyModelWrapper
from emukit.core.initial_designs import RandomDesign
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop

from experimental_design import config
from sumo_grid_simulation.grid_simulation import Simulator

## Simulator

We start by initialising our simulation.

In [3]:
simulator = Simulator(end_time=300)

We then define the two user functions in which we are interested:

In [4]:
def user_function_emissions(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function emissions called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        alpha = 0.005
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * alpha)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            maxSpeed      = maxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )

        CO2 = s['CO2']/s['num_emissions_samples']
        result.append(CO2/s['duration'])
        print(f'\nOutput {result[-1]}\n')
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

We then load the parameters' space from our configuration file:

In [5]:
parameter_space = config.get_parameter_space()

## Emulator

### Rnadomly initialised emulator

The first emulator we are going to analyse is the one that is just initilised with random points

We first sample at random 200 datapoints from the parameter space:

In [6]:
init_X, init_Y = pickle.load(open('bayesian_optimization/init_points/270_init_points_emissions.pkl', 'rb'))

'''
design = RandomDesign(parameter_space)
num_init_points = 270
init_X = design.get_samples(num_init_points)
init_Y = user_function_emissions(init_X)
init_points = init_X, init_Y
with open(f'bayesian_optimization/init_points/{num_init_points}_init_points_emissions.pkl', "wb") as f:
     pickle.dump(init_points, f)
'''

[[ 3.         13.45629089 12.62279017 31.88002303  3.          3.29806164]
 [11.         24.84758826 10.4733427  44.62352234  1.          4.43758246]
 [15.         14.38873192 16.87062442 46.33469437  1.          3.26889924]
 ...
 [10.         15.45422557 22.5955133  30.3372117   2.          4.18152089]
 [ 9.         18.66407347 22.65891601 41.91541962  3.          2.96242485]
 [15.         19.70927053 23.73218336 65.60162907  3.          3.01234321]]

User function emissions called with 270 inputs to simulate

Evaluating input: 1 of 270

 Retrying in 1 seconds

Output 320.8279111111111


Evaluating input: 2 of 270

 Retrying in 1 seconds

Output 51.16088719744498


Evaluating input: 3 of 270

 Retrying in 1 seconds

Output 62.926845939696406


Evaluating input: 4 of 270

 Retrying in 1 seconds

Output 81.64658838900145


Evaluating input: 5 of 270

 Retrying in 1 seconds

Output 39.136407752497654


Evaluating input: 6 of 270

 Retrying in 1 seconds

Output 67.52764370413911


Evaluat


Output 76.86669505821557


Evaluating input: 97 of 270

 Retrying in 1 seconds

Output 12.107707399503429


Evaluating input: 98 of 270

 Retrying in 1 seconds

Output 114.73066666666669


Evaluating input: 99 of 270

 Retrying in 1 seconds

Output 135.0811788021597


Evaluating input: 100 of 270

 Retrying in 1 seconds

Output 66.39930346527107


Evaluating input: 101 of 270

 Retrying in 1 seconds

Output 52.890672810146675


Evaluating input: 102 of 270

 Retrying in 1 seconds

Output 63.65292845338777


Evaluating input: 103 of 270

 Retrying in 1 seconds

Output 28.59618503546401


Evaluating input: 104 of 270

 Retrying in 1 seconds

Output 98.79417517006797


Evaluating input: 105 of 270

 Retrying in 1 seconds

Output 144.5017336498146


Evaluating input: 106 of 270

 Retrying in 1 seconds

Output 132.38868184134532


Evaluating input: 107 of 270

 Retrying in 1 seconds

Output 65.40056498428875


Evaluating input: 108 of 270

 Retrying in 1 seconds

Output 66.4098255669482





Output 15.773382453007859


Evaluating input: 198 of 270

 Retrying in 1 seconds

Output 28.127017978928194


Evaluating input: 199 of 270

 Retrying in 1 seconds

Output 40.504534293552815


Evaluating input: 200 of 270

 Retrying in 1 seconds

Output 78.34497676248316


Evaluating input: 201 of 270

 Retrying in 1 seconds

Output 142.066853111615


Evaluating input: 202 of 270

 Retrying in 1 seconds

Output 163.72343750000002


Evaluating input: 203 of 270

 Retrying in 1 seconds

Output 58.623671348035344


Evaluating input: 204 of 270

 Retrying in 1 seconds

Output 119.90218311427938


Evaluating input: 205 of 270

 Retrying in 1 seconds

Output 26.09636080147253


Evaluating input: 206 of 270

 Retrying in 1 seconds

Output 35.56760484301561


Evaluating input: 207 of 270

 Retrying in 1 seconds

Output 63.22352032136104


Evaluating input: 208 of 270

 Retrying in 1 seconds

Output 146.82328656929357


Evaluating input: 209 of 270

 Retrying in 1 seconds

Output 96.83490981555

To then fit a GP to these points

In [7]:
emulator = GPy.models.GPRegression(init_X, init_Y, noise_var=1e-10)
emukit_model_random_init = GPyModelWrapper(emulator, n_restarts=5)
emukit_model_random_init.optimize()

Optimization restart 1/5, f = 1297.6948600888115
Optimization restart 2/5, f = 1201.127032490439


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 3/5, f = 1202.2664897844352
Optimization restart 4/5, f = 1201.127032492374
Optimization restart 5/5, f = 1201.1270324898426


### Experimentally designed emulator with model variance

We also load the emulator obtained during experimental design:

In [8]:
emukit_model_variance = pickle.load(open('experimental_design/models/model_variance_20_init_points_250_loops_co2.pkl', 'rb'))

### Experimentally designed emulator with integrated variance reduction

We also load the emulator obtained during experimental design:

In [9]:
emukit_model_integrated_variance = pickle.load(open('experimental_design/models/integrated_variance_reduction_20_init_points_250_loops_co2.pkl', 'rb'))

## Bayesian optimisation

We now run bayesian optimisation on the emulators

In [10]:
n_iter_bo = 40

### Acquisition functions

In [11]:
acquisition_random_init = ExpectedImprovement(emukit_model_random_init)

In [12]:
acquisition_model_variance = ExpectedImprovement(emukit_model_variance)

In [13]:
acquisition_integrated_variance = ExpectedImprovement(emukit_model_integrated_variance)

### Optimisation loop

In [14]:
bo_random_init = BayesianOptimizationLoop(parameter_space, emukit_model_random_init, acquisition=acquisition_random_init)
bo_random_init.run_loop(user_function_emissions, n_iter_bo)

Optimization restart 1/5, f = 1201.1270324898426


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 2/5, f = 1201.1270324923057
Optimization restart 3/5, f = 1201.1270324894372
Optimization restart 4/5, f = 1201.1270324953118
Optimization restart 5/5, f = 1201.1270328608696
[[12. 25.  5. 70.  3.  5.]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 8.876725628276361

Optimization restart 1/5, f = 1206.0713284704955


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 2/5, f = 1444.982267500398
Optimization restart 3/5, f = 1206.0575002271908
Optimization restart 4/5, f = 1206.0575003027875
Optimization restart 5/5, f = 1206.0575002306584
[[12.         25.          5.         44.51789748  1.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 13.27183348369983

Optimization restart 1/5, f = 1211.4398420933435


...

Optimization restart 3/5, f = 1621.0380777339115
Optimization restart 4/5, f = 1355.7489772792687
Optimization restart 5/5, f = 1355.748976448252
[[15.         17.03589328  5.         61.26840156  1.          5.        ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 7.559185045990194



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1359.3319011597266
Optimization restart 2/5, f = 1359.3318803524821
Optimization restart 3/5, f = 1359.3318803344052
Optimization restart 4/5, f = 1359.3318803286124
Optimization restart 5/5, f = 1359.3318803529019
[[20.   8.   5.  70.   3.   1.5]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 5.952162774388234

Optimization restart 1/5, f = 1363.46097438045


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 2/5, f = 1363.460189392102
Optimization restart 3/5, f = 1363.4601785180737
Optimization restart 4/5, f = 1363.4601785220473
Optimization restart 5/5, f = 1363.4601785345094


In [15]:
bo_model_variance = BayesianOptimizationLoop(parameter_space, emukit_model_variance, acquisition=acquisition_model_variance)
bo_model_variance.run_loop(user_function_emissions, n_iter_bo)

Optimization restart 1/5, f = 1299.81783983801
Optimization restart 2/5, f = 1299.8178398382338
Optimization restart 3/5, f = 1342.8604332834736
Optimization restart 4/5, f = 1299.8178402451185
Optimization restart 5/5, f = 1299.817839934829
[[20.         14.40071822  5.         61.21447956  1.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 6.309442696463182



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1303.6917234756843
Optimization restart 2/5, f = 1631.0258018633267
Optimization restart 3/5, f = 1303.6916963765552
Optimization restart 4/5, f = 1303.6916963765477
Optimization restart 5/5, f = 1303.6916963815688
[[20.         25.          5.         64.32342619  1.          5.        ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 5.954431324805468

Optimization restart 1/5, f = 1307.5551846858834


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 2/5, f = 1350.2615632163324
Optimization restart 3/5, f = 1307.5551485085673
Optimization restart 4/5, f = 1307.555152236819


...

Optimization restart 2/5, f = 1438.5021956770424
Optimization restart 3/5, f = 1438.5020419525008
Optimization restart 4/5, f = 1438.502013763216
Optimization restart 5/5, f = 1442.3460731870255
[[16.          8.         20.56095533 70.          3.          5.        ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 16.151996535065773



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1442.2996197272635
Optimization restart 2/5, f = 1446.0421751429335
Optimization restart 3/5, f = 1442.2996429136385
Optimization restart 4/5, f = 1442.2996430327307
Optimization restart 5/5, f = 1442.2996427177668
[[14.          8.          5.         64.94296831  3.          5.        ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 8.532123387415917



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1445.6012416997537
Optimization restart 2/5, f = 1445.6011459246463
Optimization restart 3/5, f = 1445.6013287869628
Optimization restart 4/5, f = 1445.6012414515712
Optimization restart 5/5, f = 1449.712389537633


In [16]:
bo_integrated_variance = BayesianOptimizationLoop(parameter_space, emukit_model_integrated_variance, acquisition=acquisition_integrated_variance)
bo_integrated_variance.run_loop(user_function_emissions, n_iter_bo)

Optimization restart 1/5, f = 1281.5336113895025
Optimization restart 2/5, f = 1281.5336113959763
Optimization restart 3/5, f = 1281.5336113859976
Optimization restart 4/5, f = 1281.5336113860421
Optimization restart 5/5, f = 1281.5336113861576
[[11.         25.          5.         37.01832727  3.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 18.396061576246655



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1287.175666539175
Optimization restart 2/5, f = 1287.1625546165722
Optimization restart 3/5, f = 1287.1625544433264
Optimization restart 4/5, f = 1287.1625544475148
Optimization restart 5/5, f = 1287.1625546275231
[[20.          8.          5.         53.90881708  1.          5.        ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 7.378808786874681



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1291.3311815346433


...

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1442.0535910945086
Optimization restart 2/5, f = 1442.0535746304545
Optimization restart 3/5, f = 1442.0535744385688
Optimization restart 4/5, f = 1442.0535744384806
Optimization restart 5/5, f = 1442.0535744387325
[[20.         17.94714644  5.         63.42762328  3.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 6.657298918849945



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1445.8398842901497
Optimization restart 2/5, f = 1445.8398834357058
Optimization restart 3/5, f = 1445.8398834192333
Optimization restart 4/5, f = 1481.0911863997148
Optimization restart 5/5, f = 1445.8687045246434


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


## Results

In [27]:
parameters_name = ['Size of the grid', 'Speed limit in the net (m/s)', 'Max vehicles speed (m/s)', 'Length of the roads (m)', 'Number of lanes', 'Accelleration (m/s^2)']

### Results on the random initialised emulator

In [40]:
results_random_init = bo_random_init.get_results()

In [41]:
results_random_init_df = pd.DataFrame(results_random_init.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Max vehicles speed (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,20.0,8.0,5.0,70.0,1.0,5.0


In [42]:
results_random_init.minimum_value

5.54770372540081

### Results on the experimentally designed emulator with model variance

In [43]:
results_model_variance = bo_model_variance.get_results()

In [44]:
results_random_init_df = pd.DataFrame(results_model_variance.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Max vehicles speed (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,20.0,25.0,5.0,70.0,1.0,1.5


In [45]:
results_model_variance.minimum_value

5.469575516984683

### Results on the experimentally designed emulator with integrated variance reduction

In [46]:
results_integrated_variance = bo_integrated_variance.get_results()

In [47]:
results_random_init_df = pd.DataFrame(results_integrated_variance.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Max vehicles speed (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,20.0,25.0,5.0,70.0,1.0,1.5


In [48]:
results_integrated_variance.minimum_value

5.469575516984683

## Save stuff

In [26]:
with open(f'bayesian_optimization/bayesian_opt_results/results_random_init_emissions.pkl', "wb") as f:
     pickle.dump(results_random_init, f)

with open(f'bayesian_optimization/bayesian_opt_results/results_model_variance_emissions.pkl', "wb") as f:
     pickle.dump(results_model_variance, f)
        
with open(f'bayesian_optimization/bayesian_opt_results/results_integrated_variance_emissions.pkl', "wb") as f:
     pickle.dump(results_integrated_variance, f)